In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# Standart Libraries

import numpy as np

# Import files

import utilities
from Filter import ExtendedKalmanFilter
from Radar import Radar
from RadarSystem import RadarSystem
from Earth import Earth
from Satellite import Satellite
from SatelliteState import SatelliteState

from config import config


In [112]:
np.pi/2

1.5707963267948966

In [101]:
# Initialize Earth
earth = Earth()


# Initialize RadarSystem
# Beijing Aerospace Command and Control Center

BACC =  RadarSystem(100, Earth()) 


# Initialize Satellite
R = config['Satellite']['initial_conditions']['distance']
theta = config['Satellite']['initial_conditions']['polar_angle']
phi = config['Satellite']['initial_conditions']['azimuthal_angle']


angular_vel = 0.0010830807404
tang_vel = angular_vel * R
radial_velocity = 0
angular_velocity = 0


sat_state = SatelliteState(np.array([R, theta, phi]), np.array([0]), np.array([radial_velocity, tang_vel, angular_velocity]), np.array([0]))
tiagong = Satellite(sat_state, 0, earth=earth)

In [102]:
# Initialize the Kalman Filter
# tianhe is the chinese super computer

mean_0 = np.array([earth.re/2, np.pi/2, 1, 1, 1, 1])
cov_0 = np.eye(6)
observation_noise = np.eye(2)

tianhe = ExtendedKalmanFilter(mean_0, cov_0, earth, observation_noise=observation_noise)


In [103]:
tianhe.C

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [104]:
tianhe.m

array([[3.18906830e+06],
       [1.57079633e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00]])

In [105]:
tianhe.forecast()[0]

array([[ 3.18906909e+06],
       [ 2.07079633e+00],
       [ 3.18902989e+06],
       [ 1.50000000e+00],
       [ 1.50000000e+00],
       [-1.93936717e+04]])

In [106]:
state = np.array([sat_state.get_state_sat_plane()[0], sat_state.get_state_sat_plane()[2]])

In [107]:
tianhe.update(state)

(array([[ 3.38906856e+06],
        [ 1.00001809e+05],
        [ 3.28903209e+06],
        [ 5.00000000e-01],
        [ 1.00000000e+00],
        [-1.93936717e+04]]),
 array([[ 6.66666667e-01,  3.33333333e-01,  3.33341527e-01,
          0.00000000e+00,  0.00000000e+00, -9.05291011e-57],
        [ 3.33333333e-01,  1.66666667e+00, -3.33341527e-01,
          0.00000000e+00,  0.00000000e+00,  9.05291011e-57],
        [ 3.33341527e-01, -3.33341527e-01,  4.06806265e+13,
          0.00000000e+00,  6.37813660e+06, -1.81062652e-56],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.66666667e-01,  3.33333333e-01, -5.77406973e-50],
        [ 0.00000000e+00,  0.00000000e+00,  6.37813660e+06,
          3.33333333e-01,  1.66666667e+00, -1.15481395e-49],
        [-9.05291011e-57,  9.05291011e-57, -1.81062652e-56,
         -5.77406973e-50, -1.15481395e-49,  2.00039287e-98]]))

In [108]:
n = 10000000
simulation = tiagong.simulate(10000000)

sim_lenght = len(simulation.y[0])

predicted_states_satellite_cord = [mean_0]

In [109]:
BACC.try_detect_satellite(np.array([R, theta, phi]), 0)

[]

In [110]:
for i in range(sim_lenght):

    current_state_satellite_cord = tiagong.get_position_at_t(i)
    current_state_earth_cord = utilities.spherical_to_spherical(current_state_satellite_cord)
    noise_states_earth_cord = BACC.try_detect_satellite(current_state_earth_cord, i) # Placeholder function. Change for real one
    print(len(noise_states_earth_cord))

    if len(noise_states_earth_cord) > 0:
        for state_earth_cord in noise_states_earth_cord:
            state_satellite_cord = utilities.spherical_to_spherical(state_earth_cord)
            new_state_satellite_cord = tianhe.update(state_satellite_cord)
        
    new_state_satellite_cord = tianhe.forecast()[0]

    predicted_states_satellite_cord += new_state_satellite_cord,

#predicted_states_satellite_cord = np.array(predicted_states_satellite_cord)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


d:\Users\andre\OneDrive\Documentos\Warwick\Group Project\Satellite-Project\Filter.py:190: RuntimeWarning: overflow encountered in multiply
  self.m[2] = -self.G  * self.Me * 1/(self.m[0] ** 2) + self.m[0] * self.m[4] ** 2
d:\Users\andre\OneDrive\Documentos\Warwick\Group Project\Satellite-Project\Filter.py:195: RuntimeWarning: overflow encountered in square
  self.m[5] = -0.5 * rho * self.m[0] * (self.m[4] ** 2) * self.As * self.Cd/self.ms
d:\Users\andre\OneDrive\Documentos\Warwick\Group Project\Satellite-Project\Filter.py:74: RuntimeWarning: overflow encountered in matmul
  fore_cov = transition_matrix @ (self.C) @ transition_matrix.T + process_noise
d:\Users\andre\OneDrive\Documentos\Warwick\Group Project\Satellite-Project\Filter.py:74: RuntimeWarning: invalid value encountered in matmul
  fore_cov = transition_matrix @ (self.C) @ transition_matrix.T + process_noise
d:\Users\andre\OneDrive\Documentos\Warwick\Group Project\Satellite-Project\Filter.py:164: RuntimeWarning: overflow encou

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [87]:
len(BACC.try_detect_satellite(current_state_earth_cord, 1000))

0

In [77]:
predicted_states_satellite_cord

[array([ 3.18906909e+06,  2.07079633e+00,  3.18902989e+06,  1.50000000e+00,
         1.50000000e+00, -1.93936717e+04]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]]),
 array([[ inf],
        [ inf],
        [ inf],
        [-inf],
        [-inf],
        [-inf]])